In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def perform_stratified_sampling(sample_fraction, input_file, output_file, min_per_group=1, stratify_col='tier', random_state=42):
    """
    계층적 샘플링 수행 (기본: 티어별 샘플링).
    - sample_fraction: 각 그룹에서 비율로 뽑을 비율
    - min_per_group: 그룹이 작을 때 최소 보장 샘플 수
    - stratify_col: 계층화에 사용할 컬럼 이름
    """
    print('📂 데이터 로딩 중...')
    df = pd.read_csv(input_file)

    if stratify_col not in df.columns:
        raise ValueError(f"'{stratify_col}' 컬럼이 입력 파일에 없습니다.")

    # 그룹별로 샘플 수 계산
    def _sample_group(g):
        grp_size = len(g)
        n = max(0, int(round(grp_size * sample_fraction)))
        if n < min_per_group and grp_size > 0:
            n = min(min_per_group, grp_size)
        if n <= 0:
            return g.iloc[0:0]
        return g.sample(n=n, random_state=random_state)

    sampled_df = df.groupby(stratify_col, group_keys=False).apply(_sample_group).reset_index(drop=True)

    # 결과 출력 및 저장
    print('\n✅ 샘플링 완료!')
    print(f"전체 데이터 수: {len(df):,} 명")
    print(f"샘플링 데이터 수: {len(sampled_df):,} 명")
    print(f"샘플 내 {stratify_col} 비중:\n{(sampled_df[stratify_col].value_counts(normalize=True) * 100).round(2)}%")

    sampled_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"💾 샘플 데이터가 {output_file}에 저장되었습니다.")

In [2]:
name = 'me1_puuids_iron_diamond'
sampling = 0.5  # 0.5%
perform_stratified_sampling(
    sample_fraction = sampling / 100,
    input_file=f'{name}.csv', 
    output_file=f'{sampling}%_{name}.csv'
)

📂 데이터 로딩 중...

✅ 샘플링 완료!
전체 데이터 수: 20,830 명
샘플링 데이터 수: 103 명
샘플 내 tier 비중:
tier
GOLD        25.24
SILVER      23.30
PLATINUM    18.45
BRONZE      17.48
EMERALD      9.71
DIAMOND      3.88
IRON         1.94
Name: proportion, dtype: float64%
💾 샘플 데이터가 0.5%_me1_puuids_iron_diamond.csv에 저장되었습니다.
